In [2]:
import requests
import pandas as pd
from datetime import datetime, timedelta, timezone
from rich.console import Console
from pathlib import Path
import time

console = Console()

PHQ_TOKEN = "T5cY1FP0nY8-VBXC_GG3QGk1oxDPXmcHNAhXTs5Y"

HEADERS = {
    "Authorization": f"Bearer {PHQ_TOKEN}",
    "Accept": "application/json",
}

# According to your use case — these 3 categories matter.
CATEGORIES = "concerts,festivals,sports"

BASE_URL = "https://api.predicthq.com/v1/events/"
LIMIT = 100

# Free-tier allowed window: 90 days past, 90 days future
today = datetime.now(timezone.utc).date()
WINDOW_START = today - timedelta(days=90)
WINDOW_END = today + timedelta(days=90)

OUTPUT_DIR = Path("data/predicthq")
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)


def do_phq_request(params):
    """Robust GET with rate-limit handling, per PredictHQ docs."""
    backoff = 1
    while True:
        r = requests.get(BASE_URL, headers=HEADERS, params=params)

        # Rate limit (docs specify exponential backoff)
        if r.status_code == 429:
            console.print(f"[yellow]Rate limit hit — sleeping {backoff}s[/yellow]")
            time.sleep(backoff)
            backoff = min(backoff * 2, 30)
            continue

        if r.status_code != 200:
            console.print(f"[red]Error {r.status_code}: {r.text}[/red]")
            return None

        return r.json()


def fetch_phq_events():
    console.print("[bold cyan]Fetching PredictHQ events (concerts, festivals, sports)...[/bold cyan]")

    offset = 0
    all_results = []

    while True:
        params = {
            # Category format per docs
            "category": CATEGORIES,

            # Correct date filters (docs)
            "active.gte": WINDOW_START.isoformat(),
            "active.lte": WINDOW_END.isoformat(),

            # You want U.S. only
            "country": "US",

            # Max allowed per request
            "limit": LIMIT,
            "offset": offset,

            # Correct sort field per PredictHQ docs
            "sort": "start",
        }

        data = do_phq_request(params)
        if not data:
            break

        results = data.get("results", [])
        if not results:
            break

        all_results.extend(results)
        console.print(f"Fetched batch ({offset} → +{len(results)})")

        if len(results) < LIMIT:
            break

        offset += LIMIT

    console.print(f"[green]Total events fetched: {len(all_results)}[/green]")
    return all_results


def flatten_events(events):
    rows = []
    for e in events:
        rows.append({
            "id": e.get("id"),
            "title": e.get("title"),
            "start": e.get("start"),
            "end": e.get("end"),
            "category": e.get("category"),
            "labels": e.get("labels"),
            "rank": e.get("rank"),
            "local_rank": e.get("local_rank"),
            "phq_attendance": e.get("phq_attendance"),
            "location": e.get("location"),
            "duration": e.get("duration"),
            "place_hierarchies": e.get("place_hierarchies"),
            "updated": e.get("updated"),
        })
    return pd.DataFrame(rows)


def collect_all():
    raw = fetch_phq_events()
    df = flatten_events(raw)

    ts = datetime.now().strftime("%Y%m%d_%H%M%S")
    out_path = OUTPUT_DIR / f"phq_events_{ts}.parquet"
    df.to_parquet(out_path, index=False)

    console.print(f"[bold green]Saved → {out_path}[/bold green]")
    console.print(df.head())

    return df


if __name__ == "__main__":
    collect_all()


Fetching PredictHQ events (concerts, festivals, sports)...

Fetched batch (0 → +50)

Total events fetched: 50

Saved → data/predicthq/phq_events_20251207_054229.parquet

id                         title                 start  \
0  EjMD6djcRnZqDP8p84         Festival of Fountains  2025-05-09T13:00:00Z   
1  4Vq9nLagBjR9XgBp2g  The Blakeslee Concert Series  2025-07-11T04:00:00Z   
2  CApUtCHFeYWEkYZVuw     Ebensburg Farmer's Market  2025-07-12T04:00:00Z   
3  BD75L8EkVDNZLhotY5               Saturday Market  2025-07-12T06:00:00Z   
4  3iLeJtm7SiFZkj7sCP                   SummerStage  2025-08-01T04:00:00Z   

                    end   category labels  rank  local_rank  phq_attendance  \
0  2025-09-28T22:00:00Z  festivals   None   100         100        600000.0   
1  2025-09-27T03:59:59Z  festivals   None    50          79          1000.0   
2  2025-09-21T03:59:59Z  festivals   None    38          67           250.0   
3  2025-09-21T05:59:59Z  festivals   None    54          80          1500.0   
4  2025-10-10T03:59:59Z  festivals   None    91          89        110000.0   

                                 location  duration  \
0  [-75.6714870365859, 39.87429308556138]  12301200   
1                 [-77.917968, 33.982445]   6739199   
2                 [-78.726938, 40.485397]   6134399   
3                [-109.678224, 48.553614]   6134399   
4  [-73.97284610129972, 40.7724809320335]   6047999   

                                   place_hierarchies               updated  
0  [[6295630, 6255149, 6252001, 6254927, 5184079,...  2025-06-02T23:09:48Z  
1  [[6295630, 6255149, 6252001, 4482348, 4481757,...  2025-07-07T01:39:42Z  
2  [[6295630, 6255149, 6252001, 6254927, 5182829,...  2025-05-26T01:58:03Z  
3  [[6295630, 6255149, 6252001, 5667009, 5657400,...  2025-01-20T00:37:32Z  
4    [[6295630, 6255149, 6252001, 5128638, 5128581]]  2025-11-13T21:29:07Z

In [3]:
import requests
import pandas as pd
from rich.console import Console
from datetime import datetime, timedelta
from pathlib import Path

console = Console()

PHQ_TOKEN = "T5cY1FP0nY8-VBXC_GG3QGk1oxDPXmcHNAhXTs5Y"

headers = {
    "Authorization": f"Bearer {PHQ_TOKEN}",
    "Accept": "application/json"
}

# PredictHQ max limit is 100 per page
LIMIT = 100

CATEGORIES = ["concerts", "festivals", "sports"]

def fetch_phq_category(cat: str) -> list:
    """Fetch all events for a category with full pagination."""
    
    console.print(f"[cyan]Fetching PredictHQ events: {cat}[/cyan]")
    
    url = "https://api.predicthq.com/v1/events/"
    offset = 0
    all_events = []

    # Free plan: 90 days backward + 90 days forward
    today = datetime.utcnow().date()
    start = today - timedelta(days=90)
    end   = today + timedelta(days=90)

    while True:
        params = {
            "category": cat,
            "country": "US",
            "limit": LIMIT,
            "offset": offset,
            "start.gte": start.isoformat(),
            "start.lte": end.isoformat(),
            "sort": "start",
        }

        r = requests.get(url, headers=headers, params=params)
        
        if r.status_code != 200:
            console.print(f"[red]Error {r.status_code}: {r.text}[/red]")
            break

        data = r.json()
        results = data.get("results", [])

        console.print(f"[green]Fetched batch {offset} → +{len(results)}[/green]")

        all_events.extend(results)

        if len(results) < LIMIT:
            # Last page
            break

        offset += LIMIT

    console.print(f"[bold green]Total {cat} events: {len(all_events)}[/bold green]")

    return all_events


def flatten_event(e):
    """Flatten PredictHQ event into a clean row."""
    return {
        "id": e["id"],
        "title": e.get("title"),
        "category": e.get("category"),
        "start": e.get("start"),
        "end": e.get("end"),
        "rank": e.get("rank"),
        "local_rank": e.get("local_rank"),
        "phq_attendance": e.get("phq_attendance"),
        "location": e.get("location", []),
        "labels": e.get("labels"),
        "entities": e.get("entities"),
        "duration": e.get("duration"),
        "updated": e.get("updated"),
    }


def harvest_all():
    console.print("[bold cyan]Starting full PredictHQ harvest...[/bold cyan]")

    all_results = []

    for cat in CATEGORIES:
        results = fetch_phq_category(cat)
        all_results.extend(results)

    console.print(f"[bold green]Grand total events: {len(all_results)}[/bold green]")

    df = pd.DataFrame([flatten_event(e) for e in all_results])

    # Save
    out_dir = Path("data/predicthq")
    out_dir.mkdir(parents=True, exist_ok=True)
    
    timestamp = datetime.utcnow().strftime("%Y%m%d_%H%M%S")
    out_path = out_dir / f"phq_events_{timestamp}.parquet"

    df.to_parquet(out_path, index=False)
    console.print(f"[bold magenta]Saved → {out_path}[/bold magenta]")

    return df


if __name__ == "__main__":
    df = harvest_all()
    console.print(df.head())


Starting full PredictHQ harvest...

Fetching PredictHQ events: concerts

/tmp/ipykernel_108491/1473274799.py:31: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  today = datetime.utcnow().date()


Fetched batch 0 → +50

Total concerts events: 50

...

Total sports events: 50

Grand total events: 150

/tmp/ipykernel_108491/1473274799.py:106: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime("%Y%m%d_%H%M%S")


Saved → data/predicthq/phq_events_20251207_054509.parquet

id                                        title  category  \
0  3Ad6pUgSsDZ9YyCuGu                                    Deer Tick  concerts   
1  3NKX7YnztkZp6zvndF                                   Watchtower  concerts   
2  3NjBcgX3Hewhk4SgRe            Max Stalling and Heather Stalling  concerts   
3  3UDqEwrBU5ZZ6MPmhD  Sunday Contra at the Monday Club- September  concerts   
4  3dYkukhpwpErsSgxE5                                  Lulada Club  concerts   

                  start                   end  rank  local_rank  \
0  2025-09-08T00:00:00Z  2025-09-08T00:00:00Z    32          64   
1  2025-09-08T00:00:00Z  2025-09-08T00:00:00Z    39          62   
2  2025-09-08T00:00:00Z  2025-09-08T00:00:00Z    36          59   
3  2025-09-08T00:00:00Z  2025-09-08T02:30:00Z    39          65   
4  2025-09-08T00:00:00Z  2025-09-08T00:00:00Z    40          43   

   phq_attendance                   location labels  \
0           120.0  [-70.7040084, 42.0919348]   None   
1           287.0  [-97.3951022, 27.7977195]   None   
2           207.0        [-90.0249, 35.0611]   None   
3           289.0    [-90.355103, 38.591047]   None   
4           302.0  [-73.9573947, 40.7218813]   None   

                                            entities  duration  \
0  [{'entity_id': 'NdGqawHj2XKs2GsY7wyfj2', 'name...         0   
1  [{'entity_id': 'BWjtxGGf9XFJGdL3Ck7BP7', 'name...         0   
2  [{'entity_id': 'EWFbwCfNttFehwghTkM88w', 'name...         0   
3  [{'entity_id': '38ddfVGqFX4GH2X7ScWtEL7', 'nam...      9000   
4  [{'entity_id': 'tTYKja7c7HKygn7TYCg3Cx', 'name...         0   

                updated  
0  2025-11-18T20:12:23Z  
1  2025-09-22T00:14:57Z  
2  2025-09-22T00:09:59Z  
3  2025-09-22T00:09:59Z  
4  2025-12-05T21:57:38Z

In [4]:
import requests
import pandas as pd
from rich.console import Console
from datetime import datetime, timedelta
import time

console = Console()

PHQ_TOKEN = "T5cY1FP0nY8-VBXC_GG3QGk1oxDPXmcHNAhXTs5Y"

headers = {
    "Authorization": f"Bearer {PHQ_TOKEN}",
    "Accept": "application/json",
}

CATEGORIES = ["concerts", "festivals", "sports"]


def fetch_category(category: str, start: str, end: str, country="US") -> list:
    """
    Fetches ALL pages (up to 100) for a category.
    Returns a list of event dicts.
    """

    url = "https://api.predicthq.com/v1/events/"
    params = {
        "category": category,
        "country": country,
        "limit": 50,
        "start.gte": start,
        "start.lte": end,
        "sort": "start",   # valid
    }

    events = []

    r = requests.get(url, headers=headers, params=params)
    data = r.json()

    # First page
    events.extend(data.get("results", []))
    console.print(f"Fetched page 1 ({len(data.get('results', []))} events)")

    # Follow pagination
    page = 1
    next_url = data.get("next")

    while next_url and page < 100:
        page += 1
        console.print(f"Fetching page {page}...")

        r = requests.get(next_url, headers=headers)
        data = r.json()
        events.extend(data.get("results", []))

        next_url = data.get("next")
        time.sleep(0.2)

    console.print(f"[green]Finished category '{category}' with {len(events)} total events.[/green]")
    return events


# ---------------------------------------------------------
# MAIN HARVEST
# ---------------------------------------------------------

console.print("[bold cyan]Starting full PredictHQ harvest...[/bold cyan]")

today = datetime.utcnow().date()
start_date = str(today)
end_date = str(today + timedelta(days=90))   # trial future visibility

all_records = []

for cat in CATEGORIES:
    console.print(f"\n[bold yellow]Fetching category: {cat}[/bold yellow]")
    records = fetch_category(cat, start_date, end_date)
    all_records.extend(records)


# Convert to DataFrame
df = pd.DataFrame([
    {
        "id": e["id"],
        "title": e.get("title"),
        "start": e.get("start"),
        "end": e.get("end"),
        "category": e.get("category"),
        "labels": e.get("labels"),
        "rank": e.get("rank"),
        "local_rank": e.get("local_rank"),
        "phq_attendance": e.get("phq_attendance"),
        "location": e.get("location"),
        "updated": e.get("updated"),
    }
    for e in all_records
])

console.print(df.head())
console.print(f"[bold green]Total events fetched: {len(df)}[/bold green]")


Starting full PredictHQ harvest...

/tmp/ipykernel_108491/3441599002.py:69: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  today = datetime.utcnow().date()


Fetching category: concerts

Fetched page 1 (50 events)

Fetching page 2...

...

Fetching page 99...

Fetching page 100...

Finished category 'sports' with 5000 total events.

id                                              title  \
0  33Gk5exVx9xVfEBy55                              Songwriters Showcase!   
1  33WCpAftWkS42k6Gsp              JB Aaron and Dan Brodfuehrer at Batch   
2  36J2AoLFsui33YHRR5         Lake Effect Trombone Choir Holiday Concert   
3  38sJd7BPTQJyjJjfbo  Holiday Harmony at the Shrine with the Heartla...   
4  39jWPPKDbsWjCVj684                                           Beck-Ola   

                  start                   end  category labels  rank  \
0  2025-12-07T00:00:00Z  2025-12-07T02:00:00Z  concerts   None    37   
1  2025-12-07T00:00:00Z  2025-12-07T00:00:00Z  concerts   None    30   
2  2025-12-07T00:00:00Z  2025-12-07T00:00:00Z  concerts   None    40   
3  2025-12-07T00:00:00Z  2025-12-07T01:30:00Z  concerts   None    34   
4  2025-12-07T00:00:00Z  2025-12-07T04:00:00Z  concerts   None    28   

   local_rank  phq_attendance                          location  \
0          55           212.0            [-122.49258, 37.64962]   
1          56            96.0         [-78.9838104, 43.1078529]   
2          62           329.0         [-81.5643877, 41.4903268]   
3          75           157.0  [-96.27823219999999, 41.0751506]   
4          47            80.0          [-70.661784, 42.6123992]   

                updated  
0  2025-11-30T02:45:41Z  
1  2025-12-07T03:17:18Z  
2  2025-12-07T03:48:23Z  
3  2025-11-28T03:20:11Z  
4  2025-10-05T03:18:39Z

Total events fetched: 15000

In [ ]:
import requests
import pandas as pd
from rich.console import Console
from datetime import datetime, timedelta
import time

console = Console()

PHQ_TOKEN = "T5cY1FP0nY8-VBXC_GG3QGk1oxDPXmcHNAhXTs5Y"

headers = {
    "Authorization": f"Bearer {PHQ_TOKEN}",
    "Accept": "application/json",
}

CATEGORIES = ["concerts", "festivals", "sports"]


def fetch_all_pages(initial_url: str) -> list:
    """Fetches all pages via PredictHQ's next-link pagination."""
    events = []

    r = requests.get(initial_url, headers=headers)
    data = r.json()

    # First page
    events.extend(data.get("results", []))
    next_url = data.get("next")
    page = 1

    console.print(f"Fetched page {page}: {len(data.get('results', []))} events")

    # Additional pages — capped at 100 by plan limits
    while next_url and page < 100:
        page += 1
        console.print(f"Fetching page {page}...")

        r = requests.get(next_url, headers=headers)
        data = r.json()
        events.extend(data.get("results", []))

        next_url = data.get("next")
        time.sleep(0.2)

    return events


def fetch_category_window(category: str, start_date: str, end_date: str, country="US") -> list:
    """
    Builds an initial request URL and passes it into fetch_all_pages().
    """

    base_url = "https://api.predicthq.com/v1/events/"
    params = {
        "category": category,
        "country": country,
        "start.gte": start_date,
        "start.lte": end_date,
        "limit": 50,
        "sort": "start"  # this IS allowed
    }

    r = requests.get(base_url, headers=headers, params=params)
    data = r.json()

    # If no results or error, bail early
    if "results" not in data:
        console.print(f"[red]Error for category {category}: {data}[/red]")
        return []

    # Build the initial URL including encoded params
    initial_url = r.url

    console.print(f"[blue]Fetching {category} from {start_date} → {end_date}[/blue]")
    return fetch_all_pages(initial_url)


# ---------------------------------------------------
# MAIN HARVEST: 90 days backward + 90 days forward
# ---------------------------------------------------

console.print("[bold cyan]Starting PredictHQ (backward + forward) harvest...[/bold cyan]")

today = datetime.utcnow().date()

# trial-limited windows
back_start = str(today - timedelta(days=90))
back_end = str(today)

forward_start = str(today)
forward_end = str(today + timedelta(days=90))

console.print(f"[yellow]Backward window:[/yellow] {back_start} → {back_end}")
console.print(f"[yellow]Forward window:[/yellow]  {forward_start} → {forward_end}")

all_records = []

for cat in CATEGORIES:
    # Backward
    back_records = fetch_category_window(cat, back_start, back_end)
    console.print(f"[green]{cat} backward events: {len(back_records)}[/green]")

    # Forward
    forward_records = fetch_category_window(cat, forward_start, forward_end)
    console.print(f"[green]{cat} forward events: {len(forward_records)}[/green]")

    all_records.extend(back_records)
    all_records.extend(forward_records)

# ---------------------------------------------------
# Convert to DataFrame
# ---------------------------------------------------

df = pd.DataFrame([
    {
        "id": e["id"],
        "title": e.get("title"),
        "start": e.get("start"),
        "end": e.get("end"),
        "category": e.get("category"),
        "labels": e.get("labels"),
        "rank": e.get("rank"),
        "local_rank": e.get("local_rank"),
        "phq_attendance": e.get("phq_attendance"),
        "location": e.get("location"),
        "updated": e.get("updated"),
    }
    for e in all_records
])

console.print(df.head())
console.print(f"[bold green]TOTAL events fetched: {len(df):,}[/bold green]")

# Save it
ts = datetime.utcnow().strftime("%Y%m%d_%H%M%S")
out_path = f"data/predicthq/phq_events_full_{ts}.parquet"
df.to_parquet(out_path, index=False)

console.print(f"[bold cyan]Saved → {out_path}[/bold cyan]")


In [ ]:
import os
from pathlib import Path

list(Path("data/predicthq").glob("*.parquet"))


In [1]:
# scripts/phq_harvester.py

import requests
import pandas as pd
from rich.console import Console
from rich.status import Status
from rich.progress import Progress, SpinnerColumn, TextColumn
from datetime import datetime, timedelta
import time

console = Console()

PHQ_TOKEN = "T5cY1FP0nY8-VBXC_GG3QGk1oxDPXmcHNAhXTs5Y"

headers = {
    "Authorization": f"Bearer {PHQ_TOKEN}",
    "Accept": "application/json",
}

CATEGORIES = ["concerts", "festivals", "sports"]


# ---------------------------------------------------
# INTERNAL: SAFE REQUEST FUNCTION WITH RETRIES
# ---------------------------------------------------
def safe_get(url, params=None, retries=3):
    for attempt in range(1, retries + 1):
        try:
            r = requests.get(url, headers=headers, params=params, timeout=10)
            if r.status_code == 429:
                time.sleep(1.0)
                continue
            r.raise_for_status()
            return r.json(), r.url
        except Exception as e:
            if attempt == retries:
                console.print(f"[red]FAILED after retries → {url}[/red]")
                return None, None
            time.sleep(0.5)
    return None, None


# ---------------------------------------------------
# PAGINATION CLEAN VERSION
# ---------------------------------------------------
def fetch_all_pages(initial_url: str) -> list:
    events = []
    page = 0

    with Progress(
        SpinnerColumn(),
        TextColumn("[progress.description]{task.description}"),
        transient=True,
    ) as progress:

        task = progress.add_task("Fetching pages...", total=None)

        next_url = initial_url

        while next_url and page < 100:
            page += 1
            data, real_url = safe_get(next_url)

            if not data or "results" not in data:
                break

            batch = data.get("results", [])
            events.extend(batch)

            # throttle safely
            time.sleep(0.15)

            next_url = data.get("next")

            # Light progress update (not spam)
            if page % 3 == 0:
                progress.update(task, description=f"Fetched {len(events)} events...")

        progress.update(task, description=f"Done → {len(events)} events")

    return events


# ---------------------------------------------------
# CATEGORY WINDOW WRAPPER
# ---------------------------------------------------
def fetch_category_window(category: str, start_date: str, end_date: str, country="US") -> list:
    base_url = "https://api.predicthq.com/v1/events/"
    params = {
        "category": category,
        "country": country,
        "start.gte": start_date,
        "start.lte": end_date,
        "limit": 50,
        "sort": "start",
    }

    data, url_used = safe_get(base_url, params=params)

    if not data or "results" not in data:
        console.print(f"[red]Error fetching initial page for {category}[/red]")
        return []

    console.print(f"[cyan]→ {category}: {start_date} → {end_date}[/cyan]")
    return fetch_all_pages(url_used)


# ---------------------------------------------------
# MAIN HARVEST LOGIC
# ---------------------------------------------------
def run_harvest():
    console.print("[bold magenta]Starting PredictHQ Harvest (clean mode)...[/bold magenta]")

    today = datetime.utcnow().date()

    back_start = str(today - timedelta(days=90))
    back_end = str(today)

    forward_start = str(today)
    forward_end = str(today + timedelta(days=90))

    console.print(f"[yellow]Backward:[/yellow] {back_start} → {back_end}")
    console.print(f"[yellow]Forward:[/yellow]  {forward_start} → {forward_end}")

    all_records = []

    for cat in CATEGORIES:
        # backward
        with Status(f"[green]Fetching {cat} backward...[/green]"):
            b = fetch_category_window(cat, back_start, back_end)
        console.print(f"[green]{cat} backward: {len(b)} events[/green]")

        # forward
        with Status(f"[green]Fetching {cat} forward...[/green]"):
            f = fetch_category_window(cat, forward_start, forward_end)
        console.print(f"[green]{cat} forward: {len(f)} events[/green]")

        all_records.extend(b)
        all_records.extend(f)

    # Convert to DataFrame
    df = pd.DataFrame([
        {
            "id": e.get("id"),
            "title": e.get("title"),
            "start": e.get("start"),
            "end": e.get("end"),
            "category": e.get("category"),
            "labels": e.get("labels"),
            "rank": e.get("rank"),
            "local_rank": e.get("local_rank"),
            "phq_attendance": e.get("phq_attendance"),
            "location": e.get("location"),
            "updated": e.get("updated"),
        }
        for e in all_records
    ])

    console.print(df.head())
    console.print(f"[bold green]TOTAL events fetched: {len(df):,}[/bold green]")

    ts = datetime.utcnow().strftime("%Y%m%d_%H%M%S")
    out_path = f"data/predicthq/phq_events_full_{ts}.parquet"

    df.to_parquet(out_path, index=False)

    console.print(f"[bold cyan]Saved → {out_path}[/bold cyan]")
    return df


if __name__ == "__main__":
    run_harvest()


Starting PredictHQ Harvest (clean mode)...

/tmp/ipykernel_140678/936310770.py:114: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  today = datetime.utcnow().date()


Backward: 2025-09-08 → 2025-12-07

Forward:  2025-12-07 → 2026-03-07

Output()

...

sports forward: 5000 events

id                                        title  \
0  3Ad6pUgSsDZ9YyCuGu                                    Deer Tick   
1  3NKX7YnztkZp6zvndF                                   Watchtower   
2  3NjBcgX3Hewhk4SgRe            Max Stalling and Heather Stalling   
3  3UDqEwrBU5ZZ6MPmhD  Sunday Contra at the Monday Club- September   
4  3dYkukhpwpErsSgxE5                                  Lulada Club   

                  start                   end  category labels  rank  \
0  2025-09-08T00:00:00Z  2025-09-08T00:00:00Z  concerts   None    32   
1  2025-09-08T00:00:00Z  2025-09-08T00:00:00Z  concerts   None    39   
2  2025-09-08T00:00:00Z  2025-09-08T00:00:00Z  concerts   None    36   
3  2025-09-08T00:00:00Z  2025-09-08T02:30:00Z  concerts   None    39   
4  2025-09-08T00:00:00Z  2025-09-08T00:00:00Z  concerts   None    40   

   local_rank  phq_attendance                   location               updated  
0          64           120.0  [-70.7040084, 42.0919348]  2025-11-18T20:12:23Z  
1          62           287.0  [-97.3951022, 27.7977195]  2025-09-22T00:14:57Z  
2          59           207.0        [-90.0249, 35.0611]  2025-09-22T00:09:59Z  
3          65           289.0    [-90.355103, 38.591047]  2025-09-22T00:09:59Z  
4          43           302.0  [-73.9573947, 40.7218813]  2025-12-05T21:57:38Z

TOTAL events fetched: 30,000

/tmp/ipykernel_140678/936310770.py:162: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  ts = datetime.utcnow().strftime("%Y%m%d_%H%M%S")


Saved → data/predicthq/phq_events_full_20251207_065623.parquet